```
         Copyright Rein Halbersma 2020.
Distributed under the Boost Software License, Version 1.0.
   (See accompanying file LICENSE_1_0.txt or copy at
         http://www.boost.org/LICENSE_1_0.txt)
```

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import scrape.mga as mga

## 0) Fetching register information without search clauses:

In [2]:
register_empty = (mga.fetch_register()
    .drop_duplicates()
    .assign(CompanySeal = lambda x: x.CompanySeal.str.split('=').str[-1])
    .replace('', np.nan)
    .sort_values('CompanyName')
    .reset_index(drop=True)
)

## Fetching all menu options for the 4 search clauses:

In [3]:
licensees, statuses, services, urls = mga.fetch_menus()

## 1) Fetching register information from all licensees:

In [4]:
register_licensees = (pd
    .concat([
        mga.fetch_register(Licensee=licensee.CompanyName)
        for licensee in tqdm(licensees.itertuples(), total=licensees.shape[0])
    ])
    .drop_duplicates()
    .assign(CompanySeal = lambda x: x.CompanySeal.str.split('=').str[-1])
    .replace('', np.nan)
    .sort_values('CompanyName')
    .reset_index(drop=True)
)

100%|██████████| 406/406 [19:43&lt;00:00,  2.91s/it]


## 2) Fetching register information from all gaming service types:

In [5]:
register_services = (pd
    .concat([
        mga.fetch_register(Class=service.GamingService)
        for service in tqdm(services.itertuples(), total=services.shape[0])
    ])
    .drop_duplicates()
    .assign(CompanySeal = lambda x: x.CompanySeal.str.split('=').str[-1])
    .replace('', np.nan)
    .sort_values('CompanyName')
    .reset_index(drop=True)
)

100%|██████████| 2/2 [00:34&lt;00:00, 17.34s/it]


## 3) Fetching register information from all status types:

In [6]:
register_statuses = (pd
    .concat([
        mga.fetch_register(Status=status.Status)
        for status in tqdm(statuses.itertuples(), total=statuses.shape[0])
    ])
    .drop_duplicates()
    .assign(CompanySeal = lambda x: x.CompanySeal.str.split('=').str[-1])
    .replace('', np.nan)
    .sort_values('CompanyName')
    .reset_index(drop=True)
)

100%|██████████| 7/7 [00:48&lt;00:00,  6.88s/it]


## 4) Fetching register information from all URLs:

In [7]:
register_urls = (pd
    .concat([
        mga.fetch_register(URL=url.URL)
        for url in tqdm(urls.itertuples(), total=urls.shape[0])
    ])
    .drop_duplicates()
    .assign(CompanySeal = lambda x: x.CompanySeal.str.split('=').str[-1])
    .replace('', np.nan)
    .sort_values('CompanyName')
    .reset_index(drop=True)
)

100%|██████████| 644/644 [31:26&lt;00:00,  2.93s/it]


## Combine all unique entries in the 5 register versions found so far:

In [8]:
register = (pd
    .concat([
        register_empty,
        register_licensees,
        register_statuses,
        register_services,
        register_urls
    ])
    .drop_duplicates()
    .sort_values('CompanyName')
    .reset_index(drop=True)
)
companies = mga.eval_companies(register)

## Fetching linked companies from all companies:

In [9]:
linked_companies = (pd
    .concat([
        mga.fetch_linked_companies(company)
        for company in tqdm(companies.itertuples(), total=companies.shape[0])
    ])
    .merge(
        companies,
        how='outer',
        left_on='LinkedSeal',
        right_on='CompanySeal',
        suffixes=('', '_y'),
        indicator=True
    )
    .rename(columns={'_merge': 'LinkType'})
    # If the linked company was not present in the MGA Licensee Register, it is a daughter company.
    .replace({'LinkType': {
        'left_only': 'daughter'
    }})
    .assign(LinkType = lambda x: np.where(
        x.LinkType == 'daughter',
        x.LinkType,
        # If the linked company is not a daughter company, it is either the parent company or a partner.
        # A parent company has the same seal as the linked company, otherwise it is a partner.
        np.where(
            x.CompanySeal == x.LinkedSeal,
            'parent',
            'partner'
        )
    ))
    .drop(columns={'CompanyName_y', 'CompanySeal_y'})
    .sort_values(['CompanyName', 'LinkedName'])
    .reset_index(drop=True)
)

100%|██████████| 403/403 [18:41&lt;00:00,  2.78s/it]


## Fetching linked providers and URLs from all companies:

In [10]:
df = linked_companies.drop_duplicates(subset='LinkedSeal')
linked_providers, linked_urls = tuple(pd
    .concat(list(t), sort=False)
    .drop_duplicates()
    .reset_index(drop=True)
    # The loop over fetch_providers_and_urls returns a list of pairs of DataFrames.
    # The zip(*) operation transposes this into a pair of lists of DataFrames.
    for t in zip(*[
        mga.fetch_providers_and_urls(company)
        for company in tqdm(df.itertuples(), total=df.shape[0])
    ])
)

100%|██████████| 479/479 [37:44&lt;00:00,  4.73s/it]


## Inspect the 4 data frames:

In [11]:
register.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 621 entries, 0 to 620
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CompanyName      621 non-null    object
 1   LicenceNumber    620 non-null    object
 2   LicenceClass     621 non-null    object
 3   RegNumber        613 non-null    object
 4   CompanySeal      615 non-null    object
 5   Status           621 non-null    object
 6   TerminationDate  22 non-null     object
 7   Platform         3 non-null      object
 8   Address          528 non-null    object
 9   Telephone        535 non-null    object
 10  Email            414 non-null    object
dtypes: object(11)
memory usage: 53.5+ KB


In [12]:
register.head()

,CompanyName,LicenceNumber,LicenceClass,RegNumber,CompanySeal,Status,TerminationDate,Platform,Address,Telephone,Email
0,1x2 Network Malta Limited,MGA/CRP/635/2018,Type 1 Gaming Services,C 89109,dd45f4e9-1b07-4eb1-96f6-9b62a756bf94,Licensed,NaN,NaN,"3, Advance Business Centre,, Triq G.Flores,, S...",99552541,contact@1x2network.com
1,A.B. Europe Limited,MGA/B2C/315/2015,Type 1 Gaming Services,C 73472,b4bf3002-26a4-454a-aa54-4a6f557f50e8,Surrendered,NaN,NaN,"Ewropa Business Centre, Level 3 Suite 701,, Du...",+35625496000,service@allbets.com
2,AG Software Limited,MGA/B2B/147/2007,Type 1 Gaming Services,C 41837,0e4c58b8-3336-48df-a191-f7e2d7a0baad,Licensed,NaN,NaN,"135, High Street, Sliema, SLM 1548, Malta",+35622484901,info@aspireglobal.com
3,ASAP Italia SRL,MGA/B2C/380/2017,Type 4 Gaming Services,NaN,84550584-2198-489e-a96a-9f32218e2dc8,Cancelled,NaN,NaN,"58, Piazza di Villa Cerpegna, Roma RM, 00165, ...",NaN,NaN
4,Aha Bingo Limited,MGA/B2C/157/2008,Type 1 Gaming Services,C 43482,9abfae40-796e-4ae4-b962-76cc69d38346,Surrendered,NaN,NaN,"64a Regent House, Bisazza Street, Sliema, SLM ...",+35621313565,info@ahabingo.com


In [13]:
register.tail()

,CompanyName,LicenceNumber,LicenceClass,RegNumber,CompanySeal,Status,TerminationDate,Platform,Address,Telephone,Email
616,eSports Media Limited,MGA/B2C/239/2013,Type 2 Gaming Services,C 59418,d1a0ea97-e7ee-409e-851b-abb7ca4bf78a,Surrendered,NaN,NaN,"Suite A,, Triq Mater Boni Consigli,, Paola, PL...",+3562147483647,owenmuscat81@gmail.com
617,iGamingCloud Limited,MGA/CRP/330/2016,Type 2 Gaming Services,C 48466,14c4e0fd-c65a-4b0c-9e7d-d32828edb11e,Licensed,NaN,NaN,"@GIG Beach,, Triq id-dragunara,, St. Julians, ...",27110277,info@gig.com
618,iGamingCloud Limited,MGA/CRP/330/2016,Type 1 Gaming Services,C 48466,14c4e0fd-c65a-4b0c-9e7d-d32828edb11e,Licensed,NaN,NaN,"@GIG Beach,, Triq id-dragunara,, St. Julians, ...",27110277,info@gig.com
619,iGamingCloud Limited,MGA/CRP/330/2016/Old,Type 2 Gaming Services,C 48466,14c4e0fd-c65a-4b0c-9e7d-d32828edb11e,Licensed,NaN,NaN,"@GIG Beach,, Triq id-dragunara,, St. Julians, ...",27110277,info@gig.com
620,myLotto24 Limited,MGA/CRP/577/2018,Type 1 Gaming Services,06131579,NaN,Licensed,NaN,NaN,"Suite 1, 3rd Floor, 11-12 St. James's Square, ...",02036666100,office.lon@sg-tech.co.uk


In [14]:
linked_companies.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 576 entries, 0 to 575
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CompanyName  576 non-null    object
 1   CompanySeal  576 non-null    object
 2   LinkedName   576 non-null    object
 3   LinkedSeal   576 non-null    object
 4   LinkType     576 non-null    object
dtypes: object(5)
memory usage: 22.6+ KB


In [15]:
linked_companies.head()

,CompanyName,CompanySeal,LinkedName,LinkedSeal,LinkType
0,1x2 Network Malta Limited,dd45f4e9-1b07-4eb1-96f6-9b62a756bf94,1x2 Network Malta Limited,dd45f4e9-1b07-4eb1-96f6-9b62a756bf94,parent
1,1x2 Network Malta Limited,dd45f4e9-1b07-4eb1-96f6-9b62a756bf94,Football 1x2 Limited,4cb15f0d-8981-42c6-83ce-7f2a19980b62,daughter
2,A.B. Europe Limited,b4bf3002-26a4-454a-aa54-4a6f557f50e8,A.B. Europe Limited,b4bf3002-26a4-454a-aa54-4a6f557f50e8,parent
3,AG Software Limited,0e4c58b8-3336-48df-a191-f7e2d7a0baad,AG Software Limited,0e4c58b8-3336-48df-a191-f7e2d7a0baad,parent
4,AG Software Limited,0e4c58b8-3336-48df-a191-f7e2d7a0baad,Aspire Global International Limited,b7786557-8ced-40fe-80fe-a5cc0c72c9b0,partner


In [16]:
linked_companies.tail()

,CompanyName,CompanySeal,LinkedName,LinkedSeal,LinkType
571,bwin.party services (Malta) Limited,3cac393c-6f75-4697-9bc4-164b819a32ec,bwin.party services (Malta) Limited,3cac393c-6f75-4697-9bc4-164b819a32ec,parent
572,eGamingLab Limited,87b51d7d-88d3-4a97-aec9-c438be073e3b,eGamingLab Limited,87b51d7d-88d3-4a97-aec9-c438be073e3b,parent
573,eSports Media Limited,d1a0ea97-e7ee-409e-851b-abb7ca4bf78a,eSports Media Limited,d1a0ea97-e7ee-409e-851b-abb7ca4bf78a,parent
574,iGamingCloud Limited,14c4e0fd-c65a-4b0c-9e7d-d32828edb11e,BettingCloud Limited,4d0d00a4-f48d-44cd-a2c3-b8df41a0f560,daughter
575,iGamingCloud Limited,14c4e0fd-c65a-4b0c-9e7d-d32828edb11e,iGamingCloud Limited,14c4e0fd-c65a-4b0c-9e7d-d32828edb11e,parent


In [17]:
linked_providers.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 2585 entries, 0 to 2584
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   LinkedName       2585 non-null   object
 1   LinkedSeal       2585 non-null   object
 2   LicenceNumber    2585 non-null   object
 3   LicenceClass     2585 non-null   object
 4   ProviderName     2585 non-null   object
 5   ProviderLicence  2585 non-null   object
dtypes: object(6)
memory usage: 121.3+ KB


In [18]:
linked_providers.head()

,LinkedName,LinkedSeal,LicenceNumber,LicenceClass,ProviderName,ProviderLicence
0,1x2 Network Malta Limited,dd45f4e9-1b07-4eb1-96f6-9b62a756bf94,MGA/CRP/635/2018,Type 1 Gaming Services,Pocket Games Software Limited,MGA/B2B/435/2017
1,1x2 Network Malta Limited,dd45f4e9-1b07-4eb1-96f6-9b62a756bf94,MGA/CRP/635/2018,Type 1 Gaming Services,Oryx Gaming Limited,MGA/B2B/298/2015
2,Football 1x2 Limited,4cb15f0d-8981-42c6-83ce-7f2a19980b62,MGA/CRP/635/2018- 01,Type 1 Gaming Services,Pocket Games Software Limited,MGA/B2B/435/2017
3,Football 1x2 Limited,4cb15f0d-8981-42c6-83ce-7f2a19980b62,MGA/CRP/635/2018- 01,Type 1 Gaming Services,Oryx Gaming Limited,MGA/B2B/298/2015
4,AG Software Limited,0e4c58b8-3336-48df-a191-f7e2d7a0baad,MGA/B2B/147/2007,Type 1 Gaming Services,AG Software Limited,MGA/B2B/147/2007


In [19]:
linked_providers.tail()

,LinkedName,LinkedSeal,LicenceNumber,LicenceClass,ProviderName,ProviderLicence
2580,eGamingLab Limited,87b51d7d-88d3-4a97-aec9-c438be073e3b,MGA/B2C/221/2012,Type 2 Gaming Services,,
2581,BettingCloud Limited,4d0d00a4-f48d-44cd-a2c3-b8df41a0f560,MGA/CRP/330/2016- 01,Type 2 Gaming Services,,
2582,BettingCloud Limited,4d0d00a4-f48d-44cd-a2c3-b8df41a0f560,MGA/CRP/330/2016- 01,Type 1 Gaming Services,,
2583,iGamingCloud Limited,14c4e0fd-c65a-4b0c-9e7d-d32828edb11e,MGA/CRP/330/2016,Type 2 Gaming Services,,
2584,iGamingCloud Limited,14c4e0fd-c65a-4b0c-9e7d-d32828edb11e,MGA/CRP/330/2016,Type 1 Gaming Services,,


In [20]:
linked_urls.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   LinkedName  1265 non-null   object
 1   LinkedSeal  1265 non-null   object
 2   URL         1265 non-null   object
dtypes: object(3)
memory usage: 29.8+ KB


In [21]:
linked_urls.head()

,LinkedName,LinkedSeal,URL
0,A.B. Europe Limited,b4bf3002-26a4-454a-aa54-4a6f557f50e8,http://www.allbets.me
1,A.B. Europe Limited,b4bf3002-26a4-454a-aa54-4a6f557f50e8,http://www.allbetme.com
2,Aspire Global International Limited,b7786557-8ced-40fe-80fe-a5cc0c72c9b0,https://www.agentspinner.com/
3,Aspire Global International Limited,b7786557-8ced-40fe-80fe-a5cc0c72c9b0,https://www.queenplay.com/
4,Aspire Global International Limited,b7786557-8ced-40fe-80fe-a5cc0c72c9b0,https://www.tradacasino.com/


In [22]:
linked_urls.tail()

,LinkedName,LinkedSeal,URL
1260,eGamingLab Limited,87b51d7d-88d3-4a97-aec9-c438be073e3b,http://www.islandlotteries.com
1261,eGamingLab Limited,87b51d7d-88d3-4a97-aec9-c438be073e3b,http://www.jetbet.net
1262,eGamingLab Limited,87b51d7d-88d3-4a97-aec9-c438be073e3b,http://www.betsocial.net
1263,eGamingLab Limited,87b51d7d-88d3-4a97-aec9-c438be073e3b,http://www.dragonbet8.com
1264,eSports Media Limited,d1a0ea97-e7ee-409e-851b-abb7ca4bf78a,http://www.esportsventure.com
